In [171]:
import pandas as pd
from datetime import datetime
from calendar import monthrange

df_18 = pd.read_csv('../MIGTABO19X.csv',sep="|")
df_columns_18 = pd.read_csv('Campos MIGTAB019.csv',sep=";")
#df_18.drop('Unnamed: 3',axis=1,inplace= True)
df_18.columns = df_columns_18['NombreColumna'].tolist()

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

C:\Users\Fabian - PC\AppData\Local\Temp\ipykernel_19196\3586551123.py:5: DtypeWarning: Columns (2,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv('../MIGTABO19X.csv',sep="|")


#### Validación Campos obligatorios que no vengan nulos

In [172]:
df_columnasObligatorias = df_columns_18[df_columns_18['Obligatorio'] == 'S']['NombreColumna'].tolist()
df_Final = df_18[df_columnasObligatorias]
#df_Final.shape
nulos = df_Final.isnull().sum().reset_index()
nulos.columns = ['NombreColumna','CantidadNulos']
df_Columnas = nulos[nulos['CantidadNulos']> 0 ]['NombreColumna'].head(4).tolist()


In [173]:
errores = []

ro_operacion

In [174]:
df_18["ro_operacion"] = pd.to_numeric(df_18["ro_operacion"], errors="coerce")

validacion = df_18["ro_operacion"].isnull() | (df_18["ro_operacion"] == 0)

errores = df_18[validacion][["ro_operacion"]].assign(campo_error="ro_operacion")


ro_concepto

In [175]:
validacion = df_18["ro_concepto"].isnull() | (df_18["ro_concepto"].astype(str).str.strip() == "")

erroresValidacion = df_18[validacion][["ro_operacion"]].assign(campo_error="ro_concepto")

errores = pd.concat([errores,erroresValidacion], ignore_index=True)


ro_signo

In [176]:
# Validación 3: ro_signo
validacion =  ~df_18["ro_signo"].isin(["+", "-"]) &  df_18["ro_signo"].notna() & (df_18["ro_signo"].astype(str).str.strip() != "") 

erroresValidacion = df_18[validacion][["ro_operacion"]].assign(campo_error="ro_signo")

errores = pd.concat([errores,erroresValidacion], ignore_index=True)

Factor Rubro

In [177]:
validacion = (df_18["ro_concepto"] == "CAPITAL") & (df_18["ro_factor"] != 0)
erroresValidacion = df_18[validacion][["ro_operacion"]].assign(campo_error="ro_factor")
errores = pd.concat([errores,erroresValidacion], ignore_index=True)

Tasa Referencial

In [178]:
erroresValidacion = df_18[
    (
        (df_18["ro_concepto"] == "IMO") & 
        (df_18["ro_referencial"] != "T_USURA")
    ) | 
    (
        (df_18["ro_concepto"] == "CAP") &
        df_18["ro_referencial"].notna()
    )
][["ro_operacion"]].assign(campo_error="ro_referencial")

errores = pd.concat([errores,erroresValidacion], ignore_index=True)


ro_signo_reajuste

In [179]:
erroresValidacion = df_18[
    ~df_18["ro_signo"].isin(["+", "-","/","*"]) & 
    df_18["ro_signo"].notna() &           
    (df_18["ro_signo"].astype(str).str.strip() != "") 
][["ro_operacion"]].assign(campo_error="ro_signo_reajuste")

errores = pd.concat([errores,erroresValidacion], ignore_index=True)

ro_factor_reajuste

In [180]:
erroresValidacion = df_18[df_18["ro_concepto"].isin(["CAP","IMO"]) &
                          df_18["ro_signo_reajuste"] != 0][["ro_operacion"]].assign(campo_error="ro_factor_reajuste")

errores = pd.concat([errores,erroresValidacion], ignore_index=True)

ro_referencial_reajuste

In [181]:
erroresValidacion = df_18[df_18["ro_concepto"].isin(["CAP","IMO"]) & df_18["ro_signo_reajuste"].notna()][["ro_operacion"]].assign(campo_error="ro_referencial_reajuste")
errores = pd.concat([errores,erroresValidacion], ignore_index=True)

ro_valor

In [182]:
erroresValidacion = df_18[
    (
        df_18["ro_concepto"].isin(["INT", "IMO"]) & 
        (df_18["ro_valor"] != 0)
    ) |
    (
        (df_18["ro_concepto"] == "CAP") & 
        (df_18["ro_valor"] < 0)
    )
][["ro_operacion"]].assign(campo_error="ro_valor")

errores = pd.concat([errores,erroresValidacion], ignore_index=True)


ro_garantia

In [183]:
erroresValidacion = df_18[df_18["ro_garantia"] != 0][["ro_operacion"]].assign(campo_error="ro_garantia")
errores = pd.concat([errores,erroresValidacion], ignore_index=True)

ro_financiado

In [184]:
erroresValidacion = df_18[df_18["ro_financiado"].notna()][["ro_operacion"]].assign(campo_error="ro_financiado")

errores = pd.concat([errores,erroresValidacion], ignore_index=True)

Manejo de errores

In [185]:
df_19 = df_18[~df_18["ro_operacion"].isin(errores["ro_operacion"])]

fecha_formateada = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

In [186]:
df_19.to_csv(f'../CA_RUBRO_OP_MIG_{fecha_formateada}.txt',encoding='ISO-8859-1',  decimal=',' ,index=False, sep="|",header=False)

if not errores.empty:
    errores.to_csv('../Errores_19.csv',encoding='ISO-8859-1',  decimal=',' ,index=False)


